In [23]:
import pandas as pd
import os

# =============================================================================
# KONFIGURASI AWAL
# =============================================================================

# Base path untuk semua operasi file
BASE_PATH = r"C:\Dokumen\dishub\dashboard\JULI"  # Sesuaikan dengan path Anda

# Parameter tanggal
MINGGU1_LIST = [1, 2, 3, 4, 5, 6, 7]      # Minggu 1: Tanggal 1-7 Juli
MINGGU3_LIST = [15, 16, 17, 18, 19, 20, 21] # Minggu 3: Tanggal 15-21 Juli

# Nama checkpoint
NAMA_CHECKPOINT = [
    "diponegoro", "imam bonjol", "a yani", "gajah mada", "sudirman",
    "brantas", "patimura", "trunojoyo", "arumdalu", "mojorejo"
]

# Mapping jenis kendaraan
JENIS_MAP = {
    "Large-Sized Coach": "Bus",
    "Light Truck": "Truck",
    "Minivan": "Roda 4",
    "Pedestrian": "Pejalan kaki",
    "Pick-up Truck": "Pick-up",
    "SUV/MPV": "Roda 4",
    "Sedan": "Roda 4",
    "Tricycle": "Tossa",
    "Truck": "Truck",
    "Two Wheeler": "Sepeda motor"
}

# Mapping keterangan arah
KETERANGAN_MAP = {
    "diponegoro": "Keluar Batu",
    "imam bonjol": "Batu",
    "a yani": "Batu",
    "gajah mada": "Batu",
    "sudirman": "Keluar Batu",
    "brantas": "Masuk Batu",
    "patimura": "Masuk Batu",
    "trunojoyo": "Masuk Batu",
    "arumdalu": "Masuk Batu",
    "mojorejo": "Masuk Batu"
}

# Mapping jenis kendaraan untuk data bulanan
JENIS_MAP_BULANAN = {
    "Truk": "Truck",
    "Light Truck": "Truck",
    "Bus": "Bus",
    "Pick up Truck": "Pick-up",
    "Sedan": "Roda 4",
    "Minivan": "Roda 4",
    "SUV/MPV": "Roda 4",
    "Roda 3": "Tossa",
    "Roda 2": "Sepeda motor",
    "Pedestrian": "Pejalan kaki",
    "Unknown": "Unknown"
}

print(f"📁 Base path: {BASE_PATH}")

# =============================================================================
# TAHAP 1: FUNGSI HELPER UNTUK CLEANING
# =============================================================================

def clean_sheet_advanced(df):
    """
    Fungsi untuk cleaning sheet dengan aturan:
    1. Hapus 3 baris pertama
    2. Baris pertama setelah hapus 3 baris = header kosong, isi dengan 'No' dan 'Jenis Kendaraan'
    3. Hapus dari baris 'Vehicle Type' sampai bawah
    """
    # Hapus 3 baris pertama
    df_cleaned = df.iloc[3:].copy()
    df_cleaned = df_cleaned.reset_index(drop=True)
    
    # Cari dan hapus dari baris 'Vehicle Type' sampai bawah
    vehicle_type_row = None
    for idx, row in df_cleaned.iterrows():
        # Cek semua kolom di baris ini
        for col in df_cleaned.columns:
            cell_value = str(row[col]).strip().lower()
            if 'vehicle type' in cell_value:
                vehicle_type_row = idx
                break
        if vehicle_type_row is not None:
            break
    
    # Potong dataframe sampai sebelum baris 'Vehicle Type'
    if vehicle_type_row is not None:
        df_cleaned = df_cleaned.iloc[:vehicle_type_row]
        print(f"     🗑️  Dihapus dari baris {vehicle_type_row} (Vehicle Type) sampai bawah")
    
    # Reset index lagi setelah potong
    df_cleaned = df_cleaned.reset_index(drop=True)
    
    # Set nilai untuk 2 kolom pertama di baris pertama (header)
    if len(df_cleaned) > 0 and len(df_cleaned.columns) >= 2:
        df_cleaned.iloc[0, 0] = 'No'
        df_cleaned.iloc[0, 1] = 'Jenis Kendaraan'
    
    return df_cleaned

def dedup_columns(cols):
    """Fungsi untuk rename header duplikat"""
    counts = {}
    new_cols = []
    for col in cols:
        if col not in counts:
            counts[col] = 1
            new_cols.append(col)
        else:
            counts[col] += 1
            new_cols.append(f"{col}.{counts[col]}")
    return new_cols

# =============================================================================
# TAHAP 2: CLEANING DAN PENGGABUNGAN DATA HARIAN 2 MINGGU
# =============================================================================

def process_daily_data(tanggal_list, minggu_label):
    """Fungsi untuk memproses data harian dan menggabungkannya"""
    
    # Tahap 2.1: Cleaning individual files
    print(f"\n=== CLEANING DATA HARIAN {minggu_label} ===")
    
    for tanggal in tanggal_list:
        print(f"\n📅 Processing tanggal {tanggal} Juli...")
        
        file_path = os.path.join(BASE_PATH, f"tanggal {tanggal} juli.xlsx")
        
        try:
            # Baca semua sheet tanpa header otomatis
            dfs = pd.read_excel(file_path, sheet_name=None, header=None)
            
            # Debug: tampilkan nama sheet
            print(f"  🔍 Sheet names found: {list(dfs.keys())}")
            
            # Mapping nama sheet ke checkpoint
            mapping = {f"{i+1}. {tanggal} Juli": NAMA_CHECKPOINT[i] for i in range(10)}
            
            # Dictionary untuk menyimpan hasil cleaning per sheet
            cleaned_sheets = {}
            
            for sheet_name, df in dfs.items():
                if sheet_name in mapping:
                    print(f"  🔧 Cleaning sheet: {sheet_name}")
                    
                    # Cleaning dengan aturan baru
                    df_cleaned = clean_sheet_advanced(df)
                    
                    # Simpan dengan nama sheet asli (1. 4 Juli, 2. 4 Juli, dst)
                    cleaned_sheets[sheet_name] = df_cleaned
                    
                    print(f"     ✅ Baris setelah cleaning: {len(df_cleaned)}")
                    
                else:
                    print(f"  ⚠️  Sheet '{sheet_name}' diabaikan")
            
            # Simpan ke file Excel dengan multiple sheets (10 sheets terpisah)
            if cleaned_sheets:
                output_name = os.path.join(BASE_PATH, f"dataset_cleaned_{tanggal}juli_{minggu_label.lower()}.xlsx")
                
                with pd.ExcelWriter(output_name, engine='openpyxl') as writer:
                    for sheet_name, df_cleaned in cleaned_sheets.items():
                        # Simpan dengan nama sheet asli, tanpa index dan tanpa header default
                        df_cleaned.to_excel(writer, sheet_name=sheet_name, index=False, header=False)
                
                print(f"  💾 Saved: {output_name}")
                print(f"     📊 Total sheets: {len(cleaned_sheets)}")
                
            else:
                print(f"  ❌ No valid data for tanggal {tanggal}")
        
        except Exception as e:
            print(f"  ❌ Error processing tanggal {tanggal}: {str(e)}")
    
    # Tahap 2.2: Gabungkan menjadi dataset mingguan
    print(f"\n🔄 Memulai penggabungan dataset {minggu_label}...")
    
    weekly_df_list = []
    
    for tanggal in tanggal_list:
        print(f"\n📅 Memproses tanggal {tanggal} Juli untuk penggabungan...")
        
        file_path = os.path.join(BASE_PATH, f"dataset_cleaned_{tanggal}juli_{minggu_label.lower()}.xlsx")
        
        try:
            # Baca semua sheet dari file hasil cleaning
            dfs_cleaned = pd.read_excel(file_path, sheet_name=None, header=None)
            
            # Mapping sheet ke checkpoint
            mapping = {f"{i+1}. {tanggal} Juli": NAMA_CHECKPOINT[i] for i in range(10)}
            
            daily_df_list = []
            
            for sheet_name, df in dfs_cleaned.items():
                if sheet_name in mapping:
                    # Buat header yang proper untuk dataset gabungan
                    if len(df) > 0:
                        # Ambil baris pertama sebagai header
                        header_row = df.iloc[0].tolist()
                        
                        # Buat dataframe baru dengan header yang benar
                        data_rows = df.iloc[1:].values  # Data mulai dari baris ke-2
                        
                        # Buat dataframe dengan header dan data
                        df_proper = pd.DataFrame(data_rows, columns=header_row)
                        
                        # Tambahkan kolom Source, Tanggal, dan Minggu
                        df_proper["Source"] = mapping[sheet_name]
                        df_proper["Tanggal"] = f"{tanggal}-07-2025"
                        df_proper["Minggu"] = minggu_label
                        
                        daily_df_list.append(df_proper)
                        print(f"  ✅ Sheet {sheet_name} → {mapping[sheet_name]} ({len(df_proper)} baris)")
            
            # Gabungkan semua sheet untuk tanggal ini
            if daily_df_list:
                daily_combined = pd.concat(daily_df_list, ignore_index=True)
                weekly_df_list.append(daily_combined)
                print(f"  📊 Total baris tanggal {tanggal}: {len(daily_combined)}")
            
        except Exception as e:
            print(f"  ❌ Error memproses tanggal {tanggal}: {str(e)}")
    
    # Gabungkan semua tanggal menjadi dataset mingguan
    if weekly_df_list:
        df_mingguan = pd.concat(weekly_df_list, ignore_index=True)
        return df_mingguan
    else:
        print(f"\n❌ Tidak ada data untuk {minggu_label}")
        return pd.DataFrame()

# Proses minggu 1
print("=== MEMPROSES MINGGU 1 (1-7 Juli) ===")
df_minggu1 = process_daily_data(MINGGU1_LIST, "Minggu1")

# Proses minggu 3  
print("\n=== MEMPROSES MINGGU 3 (15-21 Juli) ===")
df_minggu3 = process_daily_data(MINGGU3_LIST, "Minggu3")

# Gabungkan 2 minggu
if not df_minggu1.empty and not df_minggu3.empty:
    df_2minggu = pd.concat([df_minggu1, df_minggu3], ignore_index=True)
    
    # Simpan data gabungan 2 minggu
    output_2minggu = os.path.join(BASE_PATH, "dataset_2minggu_gabungan.xlsx")
    df_2minggu.to_excel(output_2minggu, index=False)
    
    print(f"\n✅ Data 2 minggu tergabung: {len(df_2minggu)} baris")
    print(f"💾 Disimpan ke: {output_2minggu}")
    
else:
    print("\n❌ Error: Salah satu minggu tidak memiliki data")
    exit()

# =============================================================================
# TAHAP 3: CLEANING DAN MAPPING DATA 2 MINGGU
# =============================================================================

print("\n=== TAHAP 3: CLEANING DAN MAPPING JENIS KENDARAAN ===")

# Baca data 2 minggu yang sudah tergabung
df_2minggu = pd.read_excel(output_2minggu)

# Mapping jenis kendaraan
df_2minggu["Jenis Kendaraan"] = df_2minggu["Jenis Kendaraan"].replace(JENIS_MAP)

# Mapping Source ke keterangan arah
df_2minggu["Keterangan"] = df_2minggu["Source"].map(KETERANGAN_MAP)

# Konversi tanggal dan buat kolom Hari
df_2minggu["Tanggal"] = pd.to_datetime(df_2minggu["Tanggal"], format="%d-%m-%Y")
df_2minggu["Hari"] = df_2minggu["Tanggal"].dt.day_name()

# Ambil daftar kolom jam
jam_cols = [col for col in df_2minggu.columns if col.endswith(":00:00")]

# Hapus baris yang semua kolom jamnya bernilai 0
df_2minggu = df_2minggu[~(df_2minggu[jam_cols] == 0).all(axis=1)].reset_index(drop=True)

# Simpan kembali data yang sudah dibersihkan
df_2minggu.to_excel(output_2minggu, index=False)

print(f"✅ Data 2 minggu selesai dibersihkan")
print(f"📊 Total baris: {len(df_2minggu)}")

# =============================================================================
# TAHAP 4: ANALISIS PROPORSI PER HARI
# =============================================================================

print("\n=== TAHAP 4: ANALISIS PROPORSI PER HARI ===")

# Pisahkan data per minggu
df_minggu1_clean = df_2minggu[
    (df_2minggu["Tanggal"] >= pd.Timestamp("2025-07-01")) & 
    (df_2minggu["Tanggal"] <= pd.Timestamp("2025-07-07"))
]

df_minggu3_clean = df_2minggu[
    (df_2minggu["Tanggal"] >= pd.Timestamp("2025-07-15")) & 
    (df_2minggu["Tanggal"] <= pd.Timestamp("2025-07-21"))
]

# Gabungkan Minggu 1 & 3
df_both = pd.concat([df_minggu1_clean, df_minggu3_clean], ignore_index=True)

# Hitung rata-rata per Hari + Jenis Kendaraan + Source (Source tetap dipertahankan)
df_avg_hari = (
    df_both.groupby(["Hari", "Source", "Jenis Kendaraan", "Keterangan"], as_index=False)
    [jam_cols].mean()
)

# Tambahkan kolom Total per baris
df_avg_hari["Total"] = df_avg_hari[jam_cols].sum(axis=1)

# Hitung total per jenis kendaraan per hari (gabungan semua checkpoint)
total_per_jenis_per_hari = (
    df_avg_hari.groupby(["Hari", "Jenis Kendaraan"])["Total"]
    .sum()
    .reset_index()
    .rename(columns={"Total": "TotalJenis"})
)

# Gabungkan total ke df_avg_hari
df_proporsi = df_avg_hari.merge(total_per_jenis_per_hari, on=["Hari", "Jenis Kendaraan"])

# Hitung proporsi per checkpoint/source
df_proporsi["Proporsi"] = df_proporsi["Total"] / df_proporsi["TotalJenis"]

# Simpan hasil proporsi
proporsi_path = os.path.join(BASE_PATH, "proporsi_per_hari_2minggu.xlsx")
df_proporsi.to_excel(proporsi_path, index=False)
print(f"✅ Proporsi per checkpoint disimpan ke: {proporsi_path}")
print(f"📊 Total baris proporsi: {len(df_proporsi)}")

# =============================================================================
# TAHAP 5: PENGOLAHAN DATA BULANAN TOTAL
# =============================================================================

print("\n=== TAHAP 5: PENGOLAHAN DATA BULANAN TOTAL ===")

# Path file data bulanan
file_bulanan_path = os.path.join(BASE_PATH, "Data Volume Lalu Lintas Juli.xlsx")  # ganti nama file sesuai aslinya

# Sheet dari 1 sampai 31 (Juli)
sheet_names = [str(i) for i in range(1, 32)]
list_df = []

# Loop semua sheet
for sheet in sheet_names:
    print(f"📅 Memproses sheet: {sheet}")
    try:
        df_raw = pd.read_excel(file_bulanan_path, sheet_name=sheet, header=None)

        # Cari baris awal data (setelah header "Jenis Kendaraan")
        start_idx = df_raw[df_raw[0].astype(str).str.contains("Jenis Kendaraan", case=False, na=False)].index[0] + 1

        # Ambil header
        header_row = df_raw.iloc[start_idx - 1].fillna("NA").astype(str)
        if header_row.duplicated().any():
            print(f"    ➜ Duplikat header di sheet {sheet} ➜ auto rename")
            header_row = dedup_columns(header_row)

        # Ambil data
        df_jenis = df_raw.iloc[start_idx:].copy()
        df_jenis.columns = header_row

        # Hapus baris yang berisi arah/keterangan
        mask_arah = df_jenis.apply(
             lambda row: row.astype(str).str.contains(r"Arah|Keterangan|:", case=False, na=False).any(),
             axis=1
        )
        df_jenis = df_jenis[~mask_arah]

        # Hapus baris kosong dan total
        df_jenis = df_jenis[df_jenis["Jenis Kendaraan"].notna()]
        df_jenis = df_jenis[~df_jenis["Jenis Kendaraan"].astype(str).str.lower().str.contains("total")]

        # Tambah kolom tanggal
        df_jenis["Tanggal"] = f"{sheet}-07-2025"  # karena data Juli

        list_df.append(df_jenis)
        
    except Exception as e:
        print(f"    ❌ Error processing sheet {sheet}: {str(e)}")

print("🔄 Menggabungkan semua sheet bulanan...")
df_bulanan = pd.concat(list_df, ignore_index=True)

# Simpan hasil gabungan awal
rekap_awal_path = os.path.join(BASE_PATH, "rekap_total_jenis_kendaraan_bulanan_pertanggal.xlsx")
df_bulanan.to_excel(rekap_awal_path, index=False)
print(f"✅ File gabungan awal: {rekap_awal_path}")

# =============================================================================
# TAHAP 6: PEMBERSIHAN DATA BULANAN
# =============================================================================

print("\n=== TAHAP 6: PEMBERSIHAN DATA BULANAN ===")

# Baca kembali data bulanan
df_jenis = pd.read_excel(rekap_awal_path)

# Ubah nama kolom jam
jam_list = [f"{str(i).zfill(2)}:00:00" for i in range(24)]
columns = list(df_jenis.columns)
columns[1:25] = jam_list
df_jenis.columns = columns

# Mapping jenis kendaraan
df_jenis['Jenis Kendaraan'] = df_jenis['Jenis Kendaraan'].map(JENIS_MAP_BULANAN)

# Pastikan semua kolom jam & Total numerik
for col in jam_list + ['Total']:
    df_jenis[col] = pd.to_numeric(df_jenis[col], errors='coerce').fillna(0)

# Grouping & penjumlahan
df_jenis = df_jenis.groupby(['Tanggal', 'Jenis Kendaraan'], as_index=False)[jam_list + ['Total']].sum()

# Urutkan data
df_jenis = df_jenis.sort_values(by=['Tanggal', 'Jenis Kendaraan']).reset_index(drop=True)

# Simpan hasil akhir
rekap_bersih_path = os.path.join(BASE_PATH, "rekap_bersih_total_jenis_kendaraan_bulanan_pertanggal.xlsx")
df_jenis.to_excel(rekap_bersih_path, index=False)

print(f"✅ Rekap jenis kendaraan selesai disimpan ke: {rekap_bersih_path}")
print(f"📊 Total baris data bulanan: {len(df_jenis)}")

# =============================================================================
# TAHAP 7: ESTIMASI BULANAN PER CHECKPOINT
# =============================================================================

print("\n=== TAHAP 7: ESTIMASI BULANAN PER CHECKPOINT ===")

# Load data bulanan total
df_bulanan_final = pd.read_excel(rekap_bersih_path)

# Konversi tanggal & buat kolom Hari
df_bulanan_final["Tanggal"] = pd.to_datetime(df_bulanan_final["Tanggal"], dayfirst=True)
df_bulanan_final["Hari"] = df_bulanan_final["Tanggal"].dt.day_name()

# Ambil kolom jam
jam_cols_bulanan = [col for col in df_bulanan_final.columns if ":" in col]

# Pastikan kolom jam numeric
for col in jam_cols_bulanan:
    df_bulanan_final[col] = pd.to_numeric(df_bulanan_final[col], errors='coerce').fillna(0)

# Ubah ke long format
df_long = df_bulanan_final.melt(
    id_vars=["Tanggal", "Jenis Kendaraan", "Hari"],
    value_vars=jam_cols_bulanan,
    var_name="Jam",
    value_name="Jumlah"
)

# Load proporsi 2 minggu per checkpoint
df_proporsi_final = pd.read_excel(proporsi_path)

# Merge bulanan dengan proporsi berdasarkan Hari, Jenis Kendaraan
df_merge = df_long.merge(
    df_proporsi_final[["Hari", "Source", "Jenis Kendaraan", "Proporsi"]],
    on=["Hari", "Jenis Kendaraan"],
    how="left"
)

# Hitung estimasi jumlah per checkpoint per jam
df_merge["Jumlah_Estimasi"] = df_merge["Jumlah"] * df_merge["Proporsi"]
df_merge["Jumlah_Estimasi"] = df_merge["Jumlah_Estimasi"].fillna(0)

# Pivot kembali ke wide format
df_final = df_merge.pivot_table(
    index=["Tanggal", "Jenis Kendaraan", "Source"],
    columns="Jam",
    values="Jumlah_Estimasi",
    aggfunc="sum"
).reset_index()

# Bulatkan nilai dan isi NaN
jam_columns = [col for col in df_final.columns if ":" in col]
df_final[jam_columns] = df_final[jam_columns].fillna(0).round().astype(int)

# Hitung total per baris
df_final["Total"] = df_final[jam_columns].sum(axis=1)

# Hapus jenis kendaraan 'unknown' jika ada
df_final = df_final[df_final["Jenis Kendaraan"].str.lower() != "unknown"]

# Simpan hasil akhir
output_final_path = os.path.join(BASE_PATH, "rekap_final_estimasi_bulan_juli.xlsx")
df_final.to_excel(output_final_path, index=False)
print(f"✅ Rekap estimasi bulanan disimpan ke: {output_final_path}")
print(f"📊 Total baris estimasi final: {len(df_final)}")

# =============================================================================
# TAHAP 8: ANALISIS TAMBAHAN - JENIS KENDARAAN PER SOURCE
# =============================================================================

print("\n=== TAHAP 8: ANALISIS TAMBAHAN ===")

# Lihat jenis kendaraan unik per Source
jenis_per_source = df_2minggu.groupby("Source")["Jenis Kendaraan"].unique()

# Biar rapi jadi DataFrame
jenis_per_source_df = jenis_per_source.reset_index()
jenis_per_source_df.columns = ["Source", "Jenis Kendaraan Unik"]

print("📋 Jenis kendaraan unik per Source:")
for idx, row in jenis_per_source_df.iterrows():
    print(f"  {row['Source']}: {list(row['Jenis Kendaraan Unik'])}")

print("\n🎉 SEMUA TAHAP SELESAI!")
print(f"📁 Semua output tersimpan di: {BASE_PATH}")
print("\n📄 File output yang dihasilkan:")
print(f"   1. dataset_2minggu_gabungan.xlsx")
print(f"   2. proporsi_per_hari_2minggu.xlsx") 
print(f"   3. rekap_total_jenis_kendaraan_bulanan_pertanggal.xlsx")
print(f"   4. rekap_bersih_total_jenis_kendaraan_bulanan_pertanggal.xlsx")
print(f"   5. rekap_final_estimasi_bulan_juli.xlsx")

📁 Base path: C:\Dokumen\dishub\dashboard\JULI
=== MEMPROSES MINGGU 1 (1-7 Juli) ===

=== CLEANING DATA HARIAN Minggu1 ===

📅 Processing tanggal 1 Juli...
  🔍 Sheet names found: ['1. 1 Juli', '2. 1 Juli', '3. 1 Juli', '4. 1 Juli', '5. 1 Juli', '6. 1 Juli', '7. 1 Juli', '8. 1 Juli', '9. 1 Juli', '10. 1 Juli']
  🔧 Cleaning sheet: 1. 1 Juli
     🗑️  Dihapus dari baris 35 (Vehicle Type) sampai bawah
     ✅ Baris setelah cleaning: 35
  🔧 Cleaning sheet: 2. 1 Juli
     🗑️  Dihapus dari baris 35 (Vehicle Type) sampai bawah
     ✅ Baris setelah cleaning: 35
  🔧 Cleaning sheet: 3. 1 Juli
     🗑️  Dihapus dari baris 35 (Vehicle Type) sampai bawah
     ✅ Baris setelah cleaning: 35
  🔧 Cleaning sheet: 4. 1 Juli
     🗑️  Dihapus dari baris 35 (Vehicle Type) sampai bawah
     ✅ Baris setelah cleaning: 35
  🔧 Cleaning sheet: 5. 1 Juli
     🗑️  Dihapus dari baris 35 (Vehicle Type) sampai bawah
     ✅ Baris setelah cleaning: 35
  🔧 Cleaning sheet: 6. 1 Juli
     🗑️  Dihapus dari baris 35 (Vehicle Type) 

---------------------------

In [24]:
import pandas as pd
import os

# Paths file
BASE_PATH = r"C:\Dokumen\dishub\dashboard\JULI"  # ganti sesuai folder kamu
file_estimasi = os.path.join(BASE_PATH, "rekap_final_estimasi_bulan_juli.xlsx")
file_total_asli = os.path.join(BASE_PATH, "rekap_total_jenis_kendaraan_bulanan_pertanggal.xlsx")
output_check_path = os.path.join(BASE_PATH, "cek_estimasi_vs_total.xlsx")

# Load data
df_final = pd.read_excel(file_estimasi)
df_total = pd.read_excel(file_total_asli)

# Pastikan tanggal sama format datetime
df_final['Tanggal'] = pd.to_datetime(df_final['Tanggal'], dayfirst=True)
df_total['Tanggal'] = pd.to_datetime(df_total['Tanggal'], dayfirst=True)

# Ambil kolom jam + Total untuk dijumlahkan
jam_cols_final = [col for col in df_final.columns if ":" in col]

# Sum estimasi per tanggal + jenis kendaraan (gabungkan semua checkpoint)
df_final_sum = df_final.groupby(['Tanggal', 'Jenis Kendaraan'])[jam_cols_final + ['Total']].sum().reset_index()

# Pastikan total asli punya kolom Total numerik
df_total['Total'] = pd.to_numeric(df_total['Total'], errors='coerce').fillna(0)

# Merge untuk perbandingan
df_check = df_final_sum.merge(
    df_total[['Tanggal', 'Jenis Kendaraan', 'Total']],
    on=['Tanggal', 'Jenis Kendaraan'],
    suffixes=('_estimasi', '_asli')
)

# Hitung selisih dan persentase
df_check['Selisih'] = df_check['Total_estimasi'] - df_check['Total_asli']
df_check['Selisih_%'] = df_check['Selisih'] / df_check['Total_asli'] * 100

# Tandai jika selisih besar (>5%)
df_check['Warning'] = df_check['Selisih_%'].abs() > 5

# Simpan hasil cek
df_check.to_excel(output_check_path, index=False)

print(f"✅ Perbandingan estimasi vs total asli selesai. File tersimpan di:\n{output_check_path}")
print("⚠️ Baris dengan Warning=True artinya selisih >5% dan perlu dicek")


✅ Perbandingan estimasi vs total asli selesai. File tersimpan di:
C:\Dokumen\dishub\dashboard\JULI\cek_estimasi_vs_total.xlsx
⚠️ Baris dengan Warning=True artinya selisih >5% dan perlu dicek


.....................

In [26]:
import pandas as pd
import os

# =============================================================================
# VALIDASI KONSISTENSI DATA ESTIMASI VS TOTAL BULANAN
# =============================================================================

# Base path
BASE_PATH = r"C:\Dokumen\dishub\dashboard\JULI"  # Sesuaikan dengan path Anda

# Load data estimasi final (per checkpoint)
estimasi_path = os.path.join(BASE_PATH, "rekap_final_estimasi_bulan_juli.xlsx")
df_estimasi = pd.read_excel(estimasi_path)

# Load data total bulanan asli
total_path = os.path.join(BASE_PATH, "rekap_bersih_total_jenis_kendaraan_bulanan_pertanggal.xlsx")
df_total = pd.read_excel(total_path)

print("=== VALIDASI KONSISTENSI DATA ===")
print(f"📊 Data estimasi: {len(df_estimasi)} baris")
print(f"📊 Data total: {len(df_total)} baris")

# Konversi tanggal ke format yang sama
df_estimasi["Tanggal"] = pd.to_datetime(df_estimasi["Tanggal"])
df_total["Tanggal"] = pd.to_datetime(df_total["Tanggal"], dayfirst=True)

# Ambil kolom jam
jam_cols = [col for col in df_estimasi.columns if ":" in col]

print(f"\n📅 Kolom jam yang akan divalidasi: {len(jam_cols)} kolom")
print(f"🏢 Jumlah checkpoint dalam estimasi: {df_estimasi['Source'].nunique()}")

# =============================================================================
# VALIDASI 1: TOTAL HARIAN PER JENIS KENDARAAN
# =============================================================================

print("\n=== VALIDASI 1: TOTAL HARIAN PER JENIS KENDARAAN ===")

# Hitung total estimasi per tanggal + jenis kendaraan (gabungan semua checkpoint)
estimasi_total = df_estimasi.groupby(["Tanggal", "Jenis Kendaraan"])["Total"].sum().reset_index()
estimasi_total.columns = ["Tanggal", "Jenis Kendaraan", "Total_Estimasi"]

# Data total asli
df_total_clean = df_total[["Tanggal", "Jenis Kendaraan", "Total"]].copy()
df_total_clean.columns = ["Tanggal", "Jenis Kendaraan", "Total_Asli"]

# Merge untuk perbandingan
validasi_harian = estimasi_total.merge(df_total_clean, on=["Tanggal", "Jenis Kendaraan"], how="outer")

# Hitung selisih
validasi_harian["Selisih"] = validasi_harian["Total_Estimasi"] - validasi_harian["Total_Asli"]
validasi_harian["Selisih_Persen"] = (validasi_harian["Selisih"] / validasi_harian["Total_Asli"]) * 100

# Tampilkan hasil validasi
print(f"📊 Total kombinasi tanggal-jenis: {len(validasi_harian)}")
print(f"✅ Data yang cocok (selisih = 0): {len(validasi_harian[validasi_harian['Selisih'].abs() < 0.01])}")
print(f"⚠️  Data yang tidak cocok: {len(validasi_harian[validasi_harian['Selisih'].abs() >= 0.01])}")

# Tampilkan contoh yang tidak cocok
tidak_cocok = validasi_harian[validasi_harian['Selisih'].abs() >= 0.01].head(10)
if len(tidak_cocok) > 0:
    print(f"\n🚨 CONTOH DATA YANG TIDAK COCOK:")
    print(tidak_cocok[["Tanggal", "Jenis Kendaraan", "Total_Estimasi", "Total_Asli", "Selisih", "Selisih_Persen"]])
else:
    print(f"\n✅ SEMUA DATA COCOK SEMPURNA!")

# =============================================================================
# VALIDASI 2: TOTAL PER JAM PER JENIS KENDARAAN
# =============================================================================

print("\n=== VALIDASI 2: TOTAL PER JAM PER JENIS KENDARAAN ===")

# Hitung total estimasi per tanggal + jenis kendaraan untuk setiap jam
estimasi_jam = df_estimasi.groupby(["Tanggal", "Jenis Kendaraan"])[jam_cols].sum().reset_index()

# Merge dengan data total asli
validasi_jam = estimasi_jam.merge(df_total[["Tanggal", "Jenis Kendaraan"] + jam_cols], 
                                  on=["Tanggal", "Jenis Kendaraan"], 
                                  how="outer", suffixes=("_Estimasi", "_Asli"))

# Hitung selisih untuk setiap jam
selisih_cols = []
for jam in jam_cols:
    col_estimasi = f"{jam}_Estimasi"
    col_asli = f"{jam}_Asli"
    col_selisih = f"{jam}_Selisih"
    
    validasi_jam[col_selisih] = validasi_jam[col_estimasi] - validasi_jam[col_asli]
    selisih_cols.append(col_selisih)

# Hitung total selisih per baris
validasi_jam["Total_Selisih_Jam"] = validasi_jam[selisih_cols].sum(axis=1)

# Statistik validasi jam
total_selisih = validasi_jam["Total_Selisih_Jam"].abs().sum()
data_cocok_jam = len(validasi_jam[validasi_jam["Total_Selisih_Jam"].abs() < 0.01])

print(f"📊 Total selisih akumulatif semua jam: {total_selisih:.2f}")
print(f"✅ Data yang cocok per jam: {data_cocok_jam}/{len(validasi_jam)}")

# =============================================================================
# VALIDASI 3: CONTOH SPESIFIK TANGGAL 1
# =============================================================================

print("\n=== VALIDASI 3: CONTOH SPESIFIK TANGGAL 1 JULI ===")

tanggal_sample = "2025-07-01"
tanggal_sample_dt = pd.to_datetime(tanggal_sample)

# Data estimasi tanggal 1
estimasi_tgl1 = df_estimasi[df_estimasi["Tanggal"] == tanggal_sample_dt]

# Data total asli tanggal 1  
total_tgl1 = df_total[df_total["Tanggal"] == tanggal_sample_dt]

if len(estimasi_tgl1) > 0 and len(total_tgl1) > 0:
    print(f"\n📅 TANGGAL: {tanggal_sample}")
    
    # Contoh untuk setiap jenis kendaraan
    for jenis in estimasi_tgl1["Jenis Kendaraan"].unique():
        estimasi_jenis = estimasi_tgl1[estimasi_tgl1["Jenis Kendaraan"] == jenis]
        total_jenis = total_tgl1[total_tgl1["Jenis Kendaraan"] == jenis]
        
        if len(total_jenis) > 0:
            total_estimasi_semua_titik = estimasi_jenis["Total"].sum()
            total_asli = total_jenis["Total"].iloc[0]
            
            print(f"\n🚗 {jenis}:")
            print(f"   Estimasi (semua titik): {total_estimasi_semua_titik:.2f}")
            print(f"   Total asli: {total_asli:.2f}")
            print(f"   Selisih: {total_estimasi_semua_titik - total_asli:.2f}")
            
            # Detail per checkpoint
            print(f"   📍 Detail per checkpoint:")
            for _, row in estimasi_jenis.iterrows():
                print(f"      {row['Source']}: {row['Total']:.2f}")

# =============================================================================
# SIMPAN HASIL VALIDASI
# =============================================================================

validasi_output_path = os.path.join(BASE_PATH, "validasi_konsistensi_data.xlsx")

with pd.ExcelWriter(validasi_output_path, engine='openpyxl') as writer:
    validasi_harian.to_excel(writer, sheet_name="Validasi_Harian", index=False)
    validasi_jam.to_excel(writer, sheet_name="Validasi_Per_Jam", index=False)

print(f"\n💾 Hasil validasi disimpan ke: {validasi_output_path}")

# =============================================================================
# KESIMPULAN
# =============================================================================

print(f"\n{'='*60}")
print(f"📋 KESIMPULAN VALIDASI:")
print(f"{'='*60}")

total_data = len(validasi_harian)
data_cocok = len(validasi_harian[validasi_harian['Selisih'].abs() < 0.01])
akurasi = (data_cocok / total_data) * 100

print(f"✅ Tingkat akurasi: {akurasi:.1f}% ({data_cocok}/{total_data})")

if akurasi >= 99:
    print(f"🎉 HASIL: SANGAT BAIK! Estimasi hampir sempurna.")
elif akurasi >= 95:
    print(f"✅ HASIL: BAIK! Estimasi dapat diterima.")
elif akurasi >= 90:
    print(f"⚠️  HASIL: CUKUP! Ada beberapa ketidakcocokan.")
else:
    print(f"🚨 HASIL: PERLU PERBAIKAN! Banyak ketidakcocokan.")

print(f"{'='*60}")

=== VALIDASI KONSISTENSI DATA ===
📊 Data estimasi: 2046 baris
📊 Data total: 248 baris

📅 Kolom jam yang akan divalidasi: 24 kolom
🏢 Jumlah checkpoint dalam estimasi: 10

=== VALIDASI 1: TOTAL HARIAN PER JENIS KENDARAAN ===
📊 Total kombinasi tanggal-jenis: 248
✅ Data yang cocok (selisih = 0): 19
⚠️  Data yang tidak cocok: 198

🚨 CONTOH DATA YANG TIDAK COCOK:
      Tanggal Jenis Kendaraan  Total_Estimasi  Total_Asli  Selisih  \
0  2025-07-01             Bus          1835.0        1834      1.0   
1  2025-07-01    Pejalan kaki          1551.0        1552     -1.0   
2  2025-07-01         Pick-up          1111.0        1101     10.0   
3  2025-07-01          Roda 4         60722.0       60728     -6.0   
4  2025-07-01    Sepeda motor        141329.0      141328      1.0   
6  2025-07-01           Truck         13205.0       13211     -6.0   
8  2025-07-02             Bus          1853.0        1847      6.0   
9  2025-07-02    Pejalan kaki          1680.0        1682     -2.0   
10 2025-07

In [4]:
# Lihat jenis kendaraan unik per Source
jenis_per_source = df_2minggu.groupby("Source")["Jenis Kendaraan"].unique()

# Biar rapi jadi DataFrame
jenis_per_source_df = jenis_per_source.reset_index()
jenis_per_source_df.columns = ["Source", "Jenis Kendaraan Unik"]

print(jenis_per_source_df)


        Source                               Jenis Kendaraan Unik
0       a yani  [Truck, Sepeda motor, Roda 4, Bus, Pick-up, To...
1     arumdalu  [Truck, Sepeda motor, Roda 4, Unknown, Bus, Pi...
2      brantas  [Truck, Sepeda motor, Roda 4, Bus, Pick-up, To...
3   diponegoro  [Truck, Sepeda motor, Roda 4, Bus, Pick-up, To...
4   gajah mada  [Truck, Sepeda motor, Roda 4, Bus, Pick-up, To...
5  imam bonjol  [Truck, Sepeda motor, Roda 4, Bus, Pick-up, To...
6     mojorejo  [Truck, Sepeda motor, Roda 4, Bus, Pick-up, To...
7     patimura  [Truck, Sepeda motor, Roda 4, Bus, Pick-up, To...
8     sudirman                      [Truck, Roda 4, Bus, Pick-up]
9    trunojoyo  [Truck, Sepeda motor, Roda 4, Bus, Pick-up, To...
